# I - Importations

In [1]:
import os
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

import recommander
import importlib
importlib.reload(recommander)

<module 'recommander' from 'd:\\Formations\\Github\\Projet_10\\recommander.py'>

# Constantes

In [2]:
DATASETS_PATH = "datasets/"
ARTICLES_METADATA_PATH = DATASETS_PATH + "articles_metadata.csv"
CLICKS_PATH = DATASETS_PATH + "clicks/"
CLICKS_HOUR_CONCATENATED_PATH = DATASETS_PATH + "clicks_hour_concatenated.csv"

# Fonctions

In [3]:
def perc_fill_rate(datas: pd.DataFrame)-> float:
    """Calculer le taux de remplissage
    
    Parameters:
    datas: Dataframe dont le taux de remplissage est à calculer

    Returns:
    float: taux de remplissage calculé
    """

    return 100 - datas.isna().sum() / len(datas) * 100

def make_title(texte:str):
    """Ecrit tout simplement un ###### texte #####
    Cette fonction a été crée pour éviter de répéter les mêmes écritures de print

    Parameters:
    texte -- Texte à afficher
    """

    print("\n###### " + texte + " ######\n")

def make_plot_dtypes(datas:pd.DataFrame):
    """Crée un camembert de répartition de types utilisés de la dataframe
    
    Parameters:
    datas: Dataframe dont on veut calculer la répartition des types utilisées
    """

    print(datas.dtypes.value_counts())
    plt.figure(figsize=(10,5))
    plt.pie(datas.dtypes.value_counts().values,autopct="%1.2f%%",labels=[str(types) for types in datas.dtypes.value_counts().index])
    plt.title("Répartition des types dans le jeu de données.")
    plt.ylabel("Type des données")
    plt.legend()
    plt.show()

def make_perc_missing_values_info(datas:pd.DataFrame, perc_missing_values_threshold: int = 50) -> sns.barplot:
    """Fonction qui affiche un diagramme en barres contenant le taux de remplissage pour chaque variable de la dataframe
    
    Parameters:
    datas: Dataframe dont on désire afficher le taux de remplissage sous forme d'un diagramme en barres
    perc_missing_values_threshold: Le seuil du pourcentage qui indique si une variable est exploitable ou non (par défaut : 50)
    
    Returns:
    sns.barplot: contient l'objet de barplot
    """

    # Calcul du taux de remplissage
    perc_fill_rate_datas = perc_fill_rate(datas)
    perc_fill_rate_datas = perc_fill_rate_datas.sort_values(ascending=False)

    # Définition de la palette de couleurs en fonction du taux de remplissage
    palette = ["red" if p < perc_missing_values_threshold else "orange" if p < 100 else "green" for p in perc_fill_rate_datas.values]

    display(perc_fill_rate_datas.index)
    display(perc_fill_rate_datas.values)

    # Création d'un grpahique à barres pour le taux de remplissage
    plt.figure(figsize=(25,5))
    sns.barplot(x=perc_fill_rate_datas.index, y=perc_fill_rate_datas.values, hue=perc_fill_rate_datas.index, palette=palette)
    plt.ylabel("Taux de remplissage")
    plt.xlabel("Variables")
    plt.xticks(rotation=90)
    plt.title("Taux de remplissage du fichier")

    # Mise en place d'une ligne rouge marquant le seuil décidé %
    line_seuil = plt.axhline(y=perc_missing_values_threshold, color="red", linestyle="--", label="seuil " + str(perc_missing_values_threshold) + "%")

    # Création et paramétrage des légendes à afficher
    red_patch = mpatches.Patch(color="red", label="< " + str(perc_missing_values_threshold) + "%")
    orange_patch = mpatches.Patch(color="orange", label= "" + str(perc_missing_values_threshold) + "-]100%")
    green_patch = mpatches.Patch(color='green', label='100%')
    plt.legend(handles=[line_seuil, red_patch, orange_patch, green_patch], title='Taux de remplissage', loc='upper right')
    plt.show()

    # Afficher une dataframe sur le taux de remplissage des variables
    var_with_missing_values = perc_fill_rate_datas.loc[perc_fill_rate_datas < 100].to_frame()
    var_with_missing_values = var_with_missing_values.reset_index()
    var_with_missing_values = var_with_missing_values.rename(columns={"index": "Variable", 0: "% Taux remplissage"})
    display(var_with_missing_values)

def analyse_outlier(df: pd.DataFrame):
    """Analyse affiche un boxplot de chaque variable numérique afin d'analyser les outliers
    
    Parameters:
    df: la dataframe dont on veut analyser les outliers
    """

    # Ne sélectionner que les variables numériques
    quantitatives = df.select_dtypes(include=["number"])

    make_title("Statistiques descriptives")
    display(quantitatives.describe())

    # Afficher les boxplots de chaque variable
    nb_var = len(quantitatives.columns)
    nb_cols = 5
    nb_rows = int(nb_var/nb_cols) + 1
    fig, axes = plt.subplots(nb_rows, nb_cols, figsize=(25, nb_rows * 5))
    for i, var in enumerate(quantitatives.columns):
        if nb_rows > 1:
            ax = axes[i // nb_cols, i % nb_cols]
        else:
            ax = axes[i // nb_cols]
        sns.boxplot(x=quantitatives[var], ax=ax)

    # Supprime les emplacements non utilisés
    for i in range(nb_var, nb_rows * nb_cols):
        fig.delaxes(axes.flatten()[i])

    make_title("Boxplots des variables")
    plt.show()
    
def analyse_exploratoire_generique(datas: pd.DataFrame, perc_missing_values_threshold: int = 50, nb_index: int = 1):
    """Effectue une analyse exploratoire générique

    Parameters:
    datas: la dataframe sur laquelle on veut effectuer une analyse exploratoire générique
    perc_missing_values_threshold: Le seuil du pourcentage de valeur manquant qui indique si une variable est exploitable ou non (par défaut: 50)
    nb_index: index de début d'analyse de doublons, utilisé pour analyser les doublons en dehors des index
    """

    # Présenter les informations générales
    make_title("Informations générales")
    datas.info()

    # Afficher la répartition des types dans le jeu de données
    make_title("Répartition des types dans le dataset")
    make_plot_dtypes(datas)


    # Afficher les 5 premières et derniè_res lignes du DataFrame
    make_title("Les 5 premières et dernières lignes")
    head = datas.head()
    tail = datas.tail()
    display(head, tail)

    # Afficher le nombre de valeurs uniques
    make_title("Nombre de valeurs uniques")
    display(datas.nunique().to_frame().T)

    # Afficher le nombre des doublons génériques
    make_title("Doublons génériques (toutes les variables sont utilisées)")
    display(datas.duplicated().sum())

    # Afficher le nombre des doublons sans l'index
    make_title("Doublons sans l'index")
    display(datas[datas.columns[nb_index:]].duplicated().sum())

    # Afficher les informations sur le taux de remplissage 
    make_title("Informations du taux de remplissage")
    make_perc_missing_values_info(datas, perc_missing_values_threshold)

    # Afficher les analyses des variables numériques 
    analyse_outlier(datas)

In [4]:
def get_clicked_articles_user(user_id: int, clicks: pd.DataFrame):
    """obtenir les articles cliqués par un utilisateur donné
    
    Parameters:
    user_id: l'identifiant de l'utilisateur
    clicks: la dataframe contenant les clics
    """

    # Sélectionner les articles cliqués par l'utilisateur
    articles_user = clicks[clicks["user_id"] == user_id]
    articles_user = articles_user[["click_article_id", "click_timestamp"]]

    return articles_user

In [5]:
def get_last_clicked_articles_user(user_id: int, clicks: pd.DataFrame):
    """Obtenir le dernier article cliqué par un utilisateur

    Parameters:
    user_id: l'identifiant de l'utilisateur
    clicks: la dataframe contenant les clics
    """

    articles_user = get_clicked_articles_user(user_id, clicks)
    last_article = articles_user[articles_user["click_timestamp"] == articles_user["click_timestamp"].max()]

    return last_article
 

In [6]:
def cosine_similarity(vector1, vector2):
    """Calculer la similarité cosinus entre les articles
    """

    similarity = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
    return similarity

In [7]:
def five_most_similar_articles(article_id, articles_embeddings):
    """Trouver les 5 articles les plus similaires à un article donné
    """

    article_embedding = articles_embeddings[article_id]
    similarities = {}
    for id, embedding in enumerate(articles_embeddings):
        similarities[id] = cosine_similarity(article_embedding, embedding)
    similarities = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    return similarities[1:6]

# Corps

## Lecture des datasets

In [8]:
df_articles = pd.read_csv(ARTICLES_METADATA_PATH)

In [9]:
if os.path.isfile(CLICKS_HOUR_CONCATENATED_PATH):
    df_clicks_hour = pd.read_csv(CLICKS_HOUR_CONCATENATED_PATH)
else:
    df_clicks_hour = pd.DataFrame()

    clicks_hour_files = os.listdir(CLICKS_PATH)
    for file in clicks_hour_files:
        df_clicks_hour = pd.concat([df_clicks_hour, pd.read_csv(CLICKS_PATH + file)], ignore_index=True)
    
    df_clicks_hour.to_csv(CLICKS_HOUR_CONCATENATED_PATH, index=False)

In [75]:
with open(DATASETS_PATH + "articles_embeddings.pickle", "rb") as f:
    np_embedding_articles = pickle.load(f)


# TEST

In [11]:
recommander.most_popular_articles(df_clicks_hour, 10)

click_article_id
160974    37213
272143    28943
336221    23851
234698    23499
123909    23122
336223    21855
96210     21577
162655    21062
183176    20303
168623    19526
Name: count, dtype: int64

In [16]:
nb_clicks_ar = df_clicks_hour.groupby("click_article_id").size().sort_values(ascending=False)

In [17]:
nb_clicks_us = df_clicks_hour.groupby("user_id").size().sort_values(ascending=False)

In [33]:
ar_pas = [nb_clicks_ar[nb_clicks_ar > pas].size for pas in range(0, int(nb_clicks_ar.size / 5), 5)]
us_pas = [nb_clicks_us[nb_clicks_us > pas].size for pas in range(10, int(nb_clicks_us.size / 5), 5)]

In [34]:
resultat = np.outer(ar_pas, us_pas)

In [35]:
target = 100000000
ecarts = np.abs(resultat - target)
indice_min = np.unravel_index(np.argmin(ecarts, axis=None), ecarts.shape)

valeur_proche = resultat[indice_min]
indice_ar = indice_min[0]
indice_us = indice_min[1]

In [37]:
print(valeur_proche)
print(indice_ar * 5)
print(indice_us * 5 + 10)

99628099
270
10


In [38]:
nb_clicks_ar[nb_clicks_ar > 270].size

1289

In [39]:
nb_clicks_us[nb_clicks_us > 10].size

77291

In [40]:
test_limit_ar  = nb_clicks_ar[nb_clicks_ar > 270]
test_limit_us = nb_clicks_us[nb_clicks_us > 10]

In [65]:
df_clicks_hour[df_clicks_hour["user_id"].isin(test_limit_us.index)]["click_article_id"].nunique()

31748

In [41]:
limited_articles = df_clicks_hour[df_clicks_hour["click_article_id"].isin(test_limit_ar.index) & df_clicks_hour["user_id"].isin(test_limit_us.index)]

In [44]:
importlib.reload(recommander)
recommander.preds_svds_users([5], limited_articles, 15)

{5: click_article_id
 2647      0.202054
 3394      0.201917
 4907      0.202269
 5252      0.202343
 5292      0.203952
             ...   
 362322    0.203057
 362914    0.202396
 363173    0.202519
 363910    0.203542
 363916    0.203769
 Name: 5, Length: 1289, dtype: float64}

In [67]:
limited_articles["click_article_id"].nunique()

1289

In [76]:
limited_embedding_articles = df_embedding_articles[df_embedding_articles.index.isin(limited_articles["click_article_id"].unique())]

In [101]:
importlib.reload(recommander)
cb = recommander.best_cosine_similar_articles_per_user([5], limited_articles, df_embedding_articles)

ValueError: too many values to unpack (expected 2)

In [79]:
importlib.reload(recommander)
cb, cf = recommander.hybrid_recommander([5], limited_articles, limited_embedding_articles, 15)

KeyError: "None of [Index([156279], dtype='int64')] are in the [columns]"

In [53]:
cb

{5: 0         0.071680
 1         0.124555
 2         0.321810
 3         0.323689
 4         0.213446
             ...   
 364042    0.101237
 364043   -0.053844
 364044    0.044008
 364045    0.392420
 364046    0.007159
 Name: 5, Length: 364047, dtype: float32}

In [54]:
cf

{5: click_article_id
 2647      0.202054
 3394      0.201917
 4907      0.202269
 5252      0.202343
 5292      0.203952
             ...   
 362322    0.203057
 362914    0.202396
 363173    0.202519
 363910    0.203542
 363916    0.203769
 Name: 5, Length: 1289, dtype: float64}